# 2018 General Election Midterm Candidate Scraper

In [122]:
from urllib.request import urlopen
import re # Regex Library
import pandas as pd

In [2]:
def get_html_from_url(url): 
    return urlopen(url).read().decode("utf-8")

In [77]:
states = []
with open('states.txt', 'r') as f:
    for state in f:
        states.append(state[:-1].replace(' ', '_'))

In [78]:
states

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New_Hampshire',
 'New_Jersey',
 'New_Mexico',
 'New_York',
 'North_Carolina',
 'North_Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode_Island',
 'South_Carolina',
 'South_Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West_Virginia',
 'Wisconsin',
 'Wyoming']

## House of Representatives

### Get Ballotpedia HTML for each state

I'm encountering a problem where some lists of general election candidates are formatted differently than others. I have identified 2 patterns: a more modern html approach where "General election candidates" is between b tags and candidate names are enclosed in a tags, and an older approach where "General election candidates" is enclosed in span tags and the candidate names are in ul tags.

In [268]:
def clean_names(possible_names):
    names = []
    for string in possible_names:
        if "election" not in string and "candidate" not in string:
                    names.append(string)
    return names

In [278]:
def get_candidates_for_state(state, chamber):
    chamber_urls = {'house': 'https://ballotpedia.org/United_States_House_of_Representatives_elections_in_{},_2018', 'senate': 'https://ballotpedia.org/United_States_Senate_election_in_{},_2018'}
    html = get_html_from_url(chamber_urls[chamber].format(state))
    districts = re.findall(r"District \d{1,2}.{1,300}General election.+?Primary", html, re.DOTALL)
#     print(districts)
    data = pd.DataFrame()
    if districts:
        for district in districts:
            district_string = re.findall(r"District \d{1,2}", district)[0]
            candidate_names = clean_names(re.findall(r"<a href=.+>(.+)</a>", district))
            data = data.append(pd.DataFrame({'district': [district_string]*len(candidate_names), 'name': candidate_names, 'state': state}), ignore_index=True)
    else:
        general_candidates = re.findall(r"General election candidates.+?Primary", html, re.DOTALL)
        candidate_names = clean_names(re.findall(r"<a href=.+>(.+)</a>", general_candidates[0]))
        data = data.append(pd.DataFrame({'district': [None]*len(candidate_names), 'name': candidate_names, 'state': state}), ignore_index=True)
    return data

In [279]:
get_candidates_for_state('California', 'senate')

,district,name,state
0,None,Early voting,California
1,None,Absentee voting deadline,California
2,None,Yes,California
3,None,Same-day registration,California
4,None,Voter ID,California
5,None,Poll times,California
6,None,Recalls,California


## Get House candidates for each state

In [272]:
df = pd.DataFrame()
for state in states:
    df = df.append(get_candidates_for_state(state, 'house'), ignore_index=True)
    print(get_candidates_for_state(state))

      district                name    state
0   District 1  Robert Kennedy Jr.  Alabama
1   District 1       Bradley Byrne  Alabama
2   District 2       Tabitha Isner  Alabama
3   District 2         Martha Roby  Alabama
4   District 3       Mallory Hagan  Alabama
5   District 3         Mike Rogers  Alabama
6   District 4           Lee Auman  Alabama
7   District 4     Robert Aderholt  Alabama
8   District 5      Peter Joffrion  Alabama
9   District 5           Mo Brooks  Alabama
10  District 6        Danner Kline  Alabama
11  District 6         Gary Palmer  Alabama
12  District 7        Terri Sewell  Alabama
  district          name   state
0     None     Don Young  Alaska
1     None  Alyse Galvin  Alaska
2     None    Greg Fitch  Alaska
3     None      Sid Hill  Alaska
      district                     name    state
0   District 1       Tom O&#39;Halleran  Arizona
1   District 1             Wendy Rogers  Arizona
2   District 1              David Shock  Arizona
3   District 2         

Empty DataFrame
Columns: [district, name, state]
Index: []
  district                  name     state
0     None  Lisa Blunt Rochester  Delaware
1     None          Scott Walker  Delaware
2     None           Andrew Webb  Delaware
3     None          Marvin Davis  Delaware
4     None         Paul Johnston  Delaware
       district                      name    state
0    District 2                 Neal Dunn  Florida
1    District 2              Bob Rackleff  Florida
2    District 3                  Ted Yoho  Florida
3    District 3       Yvonne Hayes Hinson  Florida
4    District 4           John Rutherford  Florida
5    District 4               Ges Selmont  Florida
6    District 4          Joceline Berrios  Florida
7    District 4              Jason Bulger  Florida
8    District 5             Alfred Lawson  Florida
9    District 5           Virginia Fuller  Florida
10   District 6           Nancy Soderberg  Florida
11   District 6             Michael Waltz  Florida
12   District 7     

      district                name    state
0   District 1     Peter Visclosky  Indiana
1   District 1          Mark Leyva  Indiana
2   District 1   Jonathan Kleinman  Indiana
3   District 2     Jackie Walorski  Indiana
4   District 2            Mel Hall  Indiana
5   District 2        Richard Wolf  Indiana
6   District 3           Jim Banks  Indiana
7   District 3     Courtney Tritch  Indiana
8   District 4           Tobi Beck  Indiana
9   District 4           Jim Baird  Indiana
10  District 5        Susan Brooks  Indiana
11  District 5        Dee Thornton  Indiana
12  District 6   Jeannine Lee Lake  Indiana
13  District 6          Greg Pence  Indiana
14  District 6         &#91;3&#93;  Indiana
15  District 6       Heather Meloy  Indiana
16  District 6         John Miller  Indiana
17  District 7        Andre Carson  Indiana
18  District 7        Wayne Harmon  Indiana
19  District 8       Larry Bucshon  Indiana
20  District 8      William Tanoos  Indiana
21  District 9  Trey Hollingswor

     district                 name  state
0  District 1      Chellie Pingree  Maine
1  District 1        Mark Holbrook  Maine
2  District 1       Martin Grohman  Maine
3  District 2       Bruce Poliquin  Maine
4  District 2         Jared Golden  Maine
5  District 2         Tiffany Bond  Maine
6  District 2            Will Hoar  Maine
7  District 2  Dennis O&#39;Connor  Maine
8  District 2  Danielle VanHelsing  Maine
9  District 2      Henry John Bear  Maine
      district                      name     state
0   District 1               Andy Harris  Maryland
1   District 1              Jesse Colvin  Maryland
2   District 1             Jenica Martin  Maryland
3   District 2       Dutch Ruppersberger  Maryland
4   District 2                Liz Matory  Maryland
5   District 2                Guy Mimoun  Maryland
6   District 2            Michael Carney  Maryland
7   District 3             John Sarbanes  Maryland
8   District 3           Charles Anthony  Maryland
9   District 3              

     district                  name       state
0  District 1         Debra Haaland  New_Mexico
1  District 1   Janice Arnold-Jones  New_Mexico
2  District 1       Lloyd Princeton  New_Mexico
3  District 2  Xochitl Torres Small  New_Mexico
4  District 2        Yvette Herrell  New_Mexico
5  District 3         Ben Ray Lujan  New_Mexico
6  District 3   Jerald Steve McFall  New_Mexico
7  District 3   Christopher Manning  New_Mexico
        district                                               name     state
0     District 1                                         Democratic  New_York
1     District 1                                         Republican  New_York
2     District 1                                 Conservative Party  New_York
3     District 1                                        Green Party  New_York
4     District 1                                 Independence Party  New_York
5     District 1                                       Reform Party  New_York
6     District 1      

  district             name         state
0     None    Mac Schneider  North_Dakota
1     None  Kelly Armstrong  North_Dakota
2     None   Charles Tuttle  North_Dakota
       district                          name state
0    District 1                  Steve Chabot  Ohio
1    District 1                 Aftab Pureval  Ohio
2    District 1                   Dirk Kubala  Ohio
3    District 1              Mike Goldschmidt  Ohio
4    District 1                 Kiumars Kiani  Ohio
5    District 2                 Brad Wenstrup  Ohio
6    District 2                 Jill Schiller  Ohio
7    District 2              James Condit Jr.  Ohio
8    District 2                   Steve Myers  Ohio
9    District 2                   David Baker  Ohio
10   District 3                  Joyce Beatty  Ohio
11   District 3                   Jim Burgess  Ohio
12   District 3                  Millie Milam  Ohio
13   District 4                    Jim Jordan  Ohio
14   District 4                 Janet Garrett  Ohio


      district              name state
0   District 1        Rob Bishop  Utah
1   District 1      Lee Castillo  Utah
2   District 1        Adam Davis  Utah
3   District 1      Eric Eliason  Utah
4   District 2     Chris Stewart  Utah
5   District 2  Shireen Ghorbani  Utah
6   District 2   Jeffrey Whipple  Utah
7   District 2       Jan Garbett  Utah
8   District 3       John Curtis  Utah
9   District 3      James Singer  Utah
10  District 3       &#91;4&#93;  Utah
11  District 3   Gregory Duerden  Utah
12  District 3       &#91;4&#93;  Utah
13  District 4          Mia Love  Utah
14  District 4       Ben McAdams  Utah
  district             name    state
0     None      Peter Welch  Vermont
1     None       Anya Tynio  Vermont
2     None     Laura Potter  Vermont
3     None     Cris Ericson  Vermont
4     None  H. Brooke Paige  Vermont
5     None      &#91;1&#93;  Vermont
       district                   name     state
0    District 1            Rob Wittman  Virginia
1    District 1    

In [273]:
df.to_csv("candidate_names.csv")

In [274]:
df.shape

(1422, 3)